In [1]:
import pandas as pd
import numpy as np
import glob
import os

In [2]:
file_path=r"C:\Users\lilly\OneDrive\Documents\1.0_Graduate_Studies\5.0 Energy havester\5.8_code\Energy_Harvester\Processed_data_dynamic"
all_files = glob.glob(os.path.join(file_path, "*.csv")) #make list of paths
dynamic_data=[]
file_names =[]

print("loading files: ")
for file in all_files:
    file_name = os.path.splitext(os.path.basename(file))[0]  # Getting the file name without extension
    dataframe = pd.read_csv(file)
    if "GV" in file_name:
        dynamic_data.append(dataframe)
        file_names.append(file_name)
        print(file_name)
        print(dataframe.shape)

loading files: 
2019_01_15_a_GV
(317, 12)
2019_01_15_b_GV
(464, 12)
2019_01_15_c_GV
(491, 12)
2019_01_15_d_GV1
(616, 12)
2019_01_15_d_GV2
(343, 12)
2019_01_15_d_GV3
(349, 12)
2019_01_15_d_GV4
(271, 12)


In [3]:
#a
GV_offset = [4.353]
GV_start = [56]
GV_angle_change =[[[18.01,21.311,-56],[29.728, 33.0300, 56],[47.136,48.278,-16],[56.591,58.792,-40],[64.51,66.8520,40],[80.568,81.47,16],[90.515,93.817,-56],[98.254,101.557,56]]]

#b
GV_offset.append(11.941)
GV_start.append(56)
GV_angle_change.append([[4.695, 7.997, -56.0], [15.510, 18.812, 56.0], [54.680, 57.981, -56.0], [76.959, 79.301, 40.0], [126.354, 128.696, -40.0], [135.707, 139.010, 56.0], [147.340, 150.641, -56.0]])

#c
GV_offset.append(7.8887)
GV_start.append(56)
GV_angle_change.append([[7.047, 10.469, -56.0], [18.310, 21.612, 56.0], [39.609, 40.751, -20.0], [60.076, 62.417, -36.0], [70.052, 72.394, 40.0], [106.034, 108.376, -40.0], [114.205, 116.547, 40.0], [123.408, 124.430, 16.0], [130.551, 133.974, -56.0]])

#d1
GV_offset.append(13.72)
GV_start.append(0)
GV_angle_change.append([[6.478, 8.820, 40.0], [17.087, 18.109, 16.0], [34.144, 37.565, -56.0], [45.303, 46.445, 20.0], [49.832, 52.174, 36.0], [57.199, 59.541, -40.0], [88.858, 90.000, -16.0], [110.194, 113.616, 56.0], [121.835, 125.257, -56.0]])

#d2
GV_offset.append(0.0)
GV_start.append(0)
GV_angle_change.append([[10.311, 12.756, 56.0], [20.378, 22.823, -56.0], [36.178, 38.624, 56.0], [55.089, 57.533, -56.0]])

#d3
GV_offset.append(0.0)
GV_start.append(0)
GV_angle_change.append([[16.096, 17.997, 56.0], [23.501, 25.404, -56.0], [40.996, 42.298, 40.0], [62.876, 63.444, 16.0], [67.527, 68.096, -18.0], [72.960, 74.261, -38.0]])

#d4
GV_offset.append(0.0)
GV_start.append(0)
GV_angle_change.append([[7.111, 8.176, 40.0], [20.620, 21.086, 16.0], [33.091, 34.648, -56.0], [44.186, 45.743, 56.0], [61.259, 62.816, -56.0]])

In [4]:
def add_GV_angle_estimate(df,start, offset,GV_angle):
    ticks_to_ang = 1.8*18/148
    for i in GV_angle:
        i[0] += offset
        i[1] += offset
    
    df['GV'] = start*ticks_to_ang
    current_angle = start*ticks_to_ang
    segment = 0
    for i, time in enumerate(df['Time (sec)']):
        
        if segment == len(GV_angle):
            df.at[i,'GV'] = current_angle
        else:
            if time <= GV_angle[segment][0]:
                #print(current_angle)
                df.at[i,'GV'] = current_angle
            elif time < GV_angle[segment][1]:
                df.at[i,'GV'] = current_angle + (GV_angle[segment][2]*ticks_to_ang) / (GV_angle[segment][1] - GV_angle[segment][0])*(time-GV_angle[segment][0])
                
            else:
                current_angle += (GV_angle[segment][2]*ticks_to_ang)
                df.at[i,'GV'] = current_angle
                segment += 1
    
for i , df in enumerate(dynamic_data):
    add_GV_angle_estimate(df, GV_start[i], GV_offset[i], GV_angle_change[i])
    df.to_csv(file_names[i] + '.csv')